# Text scrape Leeds City Council Election results into csv 

Data will be published on Leeds data mill but if you need it sooner than two weeks you can use the following code!

In [1]:
# import library for querying website
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver

In [7]:
# get list of links for scrapping

main_page = 'https://www.leeds.gov.uk/your-council/elections/leeds-city-council-election-results'

In [9]:
# set up selenium to use chrome in headless state
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

options = Options()
options.headless = True
wd = webdriver.Chrome('../tools/chromedriver', options=options)

wd.get(main_page)

html_page = wd.page_source


/var/folders/dh/dnx6pt7d3mb4l_fs2phsjg640000gn/T/ipykernel_40726/1316070037.py:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  wd = webdriver.Chrome('../tools/chromedriver', options=options)


In [10]:
# opens page from selenium in beautiful soup
import re 

soup = BeautifulSoup(html_page, 'html.parser')

# finds specific section of page with links to each ward
accordion_layer = soup.find(id=re.compile(r"acc_[0-9]{4}"))

In [11]:
full_accordion_id_names = [tag['id'] for tag in accordion_layer.findAll(True, {'id':True})]

In [47]:
full_accordion_id_names[0]

'acc_1381_panel_1_trigger'

In [52]:
accordion_layer.find(id=full_accordion_id_names[0]).contents

['\tAdel & Wharfedale\t']

In [28]:
tbl_list = pd.read_html(accordion_layer.find(id=full_accordion_id_names[1]).prettify())

In [46]:
meta_tbl = tbl_list[1].T

meta_tbl.columns = meta_tbl.loc[0,:]

meta_tbl.columns = meta_tbl.columns.str.replace(":","")

meta_tbl.drop(0, axis=0, inplace=True)

for col in meta_tbl.columns:
    tbl_list[0][col] = meta_tbl[col].values[0]

tbl_list[0]

,Candidate,Description,Votes,Electorate,Turnout,Spoilt ballots
0,ANDERSON Barry John,The Conservative Party Candidate,4385,16472,47.3%,28
1,BRIDGES Chris,Labour Party,2104,16472,47.3%,28
2,LOVE Fiona Sarah Heather,Green Party,486,16472,47.3%,28
3,ROBINSON Corey,Yorkshire Party,127,16472,47.3%,28
4,TWITCHETT Mark John,Liberal Democrats,663,16472,47.3%,28


In [58]:
frame_stack = []

for layer in full_accordion_id_names:

    if "trigger" in layer:
        
        ward = accordion_layer.find(id=layer).contents[0].strip()

    else:
        tbl_list = pd.read_html(accordion_layer.find(id=layer).prettify())

        meta_tbl = tbl_list[1].T

        main_tbl = tbl_list[0]

        meta_tbl.columns = meta_tbl.loc[0,:]

        meta_tbl.columns = meta_tbl.columns.str.replace(":","")

        meta_tbl.drop(0, axis=0, inplace=True)

        for col in meta_tbl.columns:
            main_tbl[col] = meta_tbl[col].values[0]

        main_tbl['Ward'] = ward

        frame_stack.append(main_tbl)

results_frame = pd.concat(frame_stack)
        


,Candidate,Description,Votes,Electorate,Turnout,Spoilt ballots,Ward
0,ANDERSON Barry John,The Conservative Party Candidate,4385,16472,47.3%,28,Adel & Wharfedale
1,BRIDGES Chris,Labour Party,2104,16472,47.3%,28,Adel & Wharfedale
2,LOVE Fiona Sarah Heather,Green Party,486,16472,47.3%,28,Adel & Wharfedale
3,ROBINSON Corey,Yorkshire Party,127,16472,47.3%,28,Adel & Wharfedale
4,TWITCHETT Mark John,Liberal Democrats,663,16472,47.3%,28,Adel & Wharfedale
...,...,...,...,...,...,...,...
4,WILSON Izaak William,Labour and Co-operative Party,3331,15982,42.6%,19,Weetwood
0,DIXON-MURROW Luke,Labour Party,878,16640,44.9%,29,Wetherby
1,HARRINGTON Norma Agnes,The Conservative Party Candidate,3514,16640,44.9%,29,Wetherby
2,MCINTEE Lesley Ann,Liberal Democrats,484,16640,44.9%,29,Wetherby
